In [1]:
import pickle

import numpy as np
from tqdm import tqdm
import torch

from ela import generator

import utils
import metrics
import datasets


experiments = [
    'nopoolrefinenet_dpn92_dual_hypercolumn_poly_lr_aux_data_pseudo_labels',
]

test_predictions_experiment = []

for name in experiments:
    test_predictions = utils.TestPredictions('{}'.format(name), mode='val')
    test_predictions_experiment.append(test_predictions.load_raw())

samples_train = utils.get_train_samples()
transforms = generator.TransformationsGenerator([])
dataset = datasets.AnalysisDataset(samples_train, './data/train', transforms, utils.TestPredictions('{}'.format(name), mode='val').load())

train = {}
for id in tqdm(samples_train):
    _, mask, _ = dataset.get_by_id(id)
    test_prediction = np.concatenate([predictions[id] for predictions in test_predictions_experiment], axis=0)
    prediction = torch.mean(torch.sigmoid(torch.FloatTensor(test_prediction)), dim=0)
    mask = torch.FloatTensor(mask)

    train[id] = (prediction, mask)


def strip_nan(e):
    if e != e:
        return None

    return e[:-4]


with open('./data/8_neighbours_mosaics.pkl', "rb") as f:
    neighbors = pickle.load(f)
    neighbors = {k[:-4]: [strip_nan(e) for e in v] for k, v in neighbors.items()}

samples_test = utils.get_test_samples()
masks_test = utils.TestPredictions('{}-split_{}'.format(name, 0)).load()

100%|██████████| 4000/4000 [00:05<00:00, 708.17it/s]                                                                                                


In [28]:
id_dataset = datasets.IdDataset(samples_train, './data/train')

def get_mask(id):
    if id in samples_train:
        mask_n = id_dataset[id]

    if id in samples_test:
        mask_n = torch.FloatTensor(masks_test[id])
        
    return mask_n

train_postprocessed = {id: (prediction.clone(), mask.clone()) for id, (prediction, mask) in train.items()}
for sample in tqdm(samples_train):
    if sample in neighbors:
        sample_neighbors = neighbors[sample]
        prediction_sample, mask_sample = train_postprocessed[sample]
        
        n_top, n_left, n_right, n_bottom =  sample_neighbors[1],  sample_neighbors[3],  sample_neighbors[4],  sample_neighbors[6]
        if n_top and n_left and n_right and n_bottom:
            top = get_mask(n_top)
            left = get_mask(n_left)
            right = get_mask(n_right)
            bottom = get_mask(n_bottom)
            
            if torch.mean(top) > 0.7 and torch.mean(left) > 0.7 and torch.mean(right) > 0.7 and torch.mean(bottom) > 0.7:
                prediction_sample[:, :] = 1
                train_postprocessed[sample] = prediction_sample, mask_sample
            
            #print(torch.mean(prediction_sample), torch.mean(top), torch.mean(left), torch.mean(right), torch.mean(bottom))
        
        masks_neighbors = []
        for n in sample_neighbors:
            if n in samples_train:
                mask_n = train[n][0]

            if n in samples_test:
                mask_n = torch.FloatTensor(masks_test[n])

            masks_neighbors.append(mask_n)

        if len(masks_neighbors) == 0:
            continue

        num_neighbors = len(masks_neighbors)
        masks_neighbors = torch.cat(masks_neighbors, dim=0)
        mean_neighbors = torch.mean(masks_neighbors)



predictions = [prediction for id, (prediction, mask) in train_postprocessed.items()]
predictions = torch.stack(predictions, dim=0).cuda(1)
masks = [mask for id, (prediction, mask) in train_postprocessed.items()]
masks = torch.stack(masks, dim=0).cuda(1)

predictions = (predictions > 0.5).float()

map = metrics.mAP(predictions, masks)

print('mAP', map)


100%|██████████| 4000/4000 [00:08<00:00, 472.60it/s]


mAP tensor(0.8736, device='cuda:1')
